In [1]:
# Python program to create Blockchain with Transactions

import datetime
import hashlib
import json
from flask import Flask, jsonify, request

class Blockchain:
    def __init__(self):
        self.chain = []
        self.transactions = []  # ✅ Added: list to store pending transactions
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'transactions': self.transactions  # ✅ Added: transactions to the block
        }
        self.transactions = []  # ✅ Clear the transactions list after adding to the block
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        new_proof = 1
        check_proof = False

        while not check_proof:
            hash_operation = hashlib.sha256(
                str(new_proof**2 - previous_proof**2).encode()).hexdigest()
            if hash_operation[:5] == '00000':
                check_proof = True
            else:
                new_proof += 1

        return new_proof

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1

        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False

            previous_proof = previous_block['proof']
            proof = block['proof']
            hash_operation = hashlib.sha256(
                str(proof**2 - previous_proof**2).encode()).hexdigest()

            if hash_operation[:5] != '00000':
                return False

            previous_block = block
            block_index += 1

        return True

    def add_transaction(self, sender, recipient, amount):
        """✅ Add a transaction to the list of pending transactions"""
        self.transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount
        })
        return self.get_previous_block()['index'] + 1  # Index of block that will hold this transaction

# Creating the Web App
app = Flask(__name__)

# Creating the Blockchain
blockchain = Blockchain()

# Mining a new block
@app.route('/mine_block', methods=['GET'])
def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)

    response = {
        'message': 'A block has been mined!',
        'index': block['index'],
        'timestamp': block['timestamp'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
        'transactions': block['transactions']
    }
    return jsonify(response), 200

# Getting the full Blockchain
@app.route('/get_chain', methods=['GET'])
def get_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain)
    }
    return jsonify(response), 200

# Checking if Blockchain is valid
@app.route('/is_valid', methods=['GET'])
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    response = {'message': 'The Blockchain is valid.'} if is_valid else {'message': 'The Blockchain is not valid.'}
    return jsonify(response), 200

# Adding a new transaction
@app.route('/add_transaction', methods=['POST'])
def add_transaction():
    json_data = request.get_json()
    transaction_keys = ['sender', 'recipient', 'amount']
    if not all(key in json_data for key in transaction_keys):
        return 'Some elements of the transaction are missing', 400

    index = blockchain.add_transaction(
        sender=json_data['sender'],
        recipient=json_data['recipient'],
        amount=json_data['amount']
    )
    response = {'message': f'This transaction will be added to Block {index}'}
    return jsonify(response), 201

# Running the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.22.250.39:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2025 11:50:24] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2025 11:50:30] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:50:52] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:51:21] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:53:29] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [03/Jun/2025 11:53:41] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:53:43] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:53:52] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:53:54] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:54:04] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:54:05] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:54:49] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:57:34] "G